# Fiona and Shapely

[Fiona](http://toblerity.org/fiona/) is build on top of OGR and it introduces the object-oriented approach to the work with vector data. The data are re-mapped in to [GeoJSON](http://geojson.org) structure. The interaction is more straight-forward compared to OGR.

Fiona is not suited to be the best for all tasks - simplicity is buyed back by the fact, all data need to be loaded in memmory. Also speed can be lower, when dealing with larger amounts of data. Where OGR uses pointers, Fiona has to deal with whole python objects. Generally: if you need to filter vector features, OGR should be faster, if you need to work with features sequentially, Fiona should be better.

**Note**:
> Fiona is great for binary data formats. For JSON, use the `json` python package. For data stored in database, use standard Python database interface.

[Shapely](http://toblerity.org/shapely/manual.html) Shapely is a Python package for set-theoretic analysis and manipulation of planar features using (via Python’s ctypes module) functions from the well known and widely deployed GEOS library. 

**Short version:** 
> Fiona is here for attribute and whole data source manipulation, Shapely just manupulates vector data geometries.

## Open data source

First we need to create so called collection of features:

In [5]:
import fiona
chko = fiona.open('../../data/protected_areas-etrs.shp', 'r') # collection - data are stored in ../../data directory 
chko

<open Collection '../../data/natural.shp:natural', mode 'r' at 0x7f63041374a8>

We can now mine some metadata about this feature collection, see [documentation](http://toblerity.org/fiona/manual.html)

In [6]:
# driver
chko.driver

'ESRI Shapefile'

In [8]:
# coordinate reference system
chko.crs

{'init': 'epsg:4326'}

In [9]:
# file name
chko.path

'../../data/natural.shp'

In [10]:
# layer name
chko.name

'natural'

In [11]:
# bounding box coordinates
chko.bounds

(12.0223154, 48.52299, 18.8836457, 51.0767239)

In [12]:
# everything in one step
print(chko.meta)

{'schema': {'geometry': 'Polygon', 'properties': OrderedDict([('osm_id', 'str:11'), ('name', 'str:48'), ('type', 'str:16')])}, 'driver': 'ESRI Shapefile', 'crs_wkt': 'GEOGCS["GCS_WGS_1984",DATUM["WGS_1984",SPHEROID["WGS_84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295],AUTHORITY["EPSG","4326"]]', 'crs': {'init': 'epsg:4326'}}


In [15]:
# try better format
import json
print(json.dumps(chko.meta, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "crs": {
        "init": "epsg:4326"
    },
    "crs_wkt": "GEOGCS[\"GCS_WGS_1984\",DATUM[\"WGS_1984\",SPHEROID[\"WGS_84\",6378137,298.257223563]],PRIMEM[\"Greenwich\",0],UNIT[\"Degree\",0.017453292519943295],AUTHORITY[\"EPSG\",\"4326\"]]",
    "driver": "ESRI Shapefile",
    "schema": {
        "geometry": "Polygon",
        "properties": {
            "name": "str:48",
            "osm_id": "str:11",
            "type": "str:16"
        }
    }
}


## Features in collections
Features within opened collection can be iterated

In [21]:
print(len(chko))

96115


## Coordinate reference systems
Fiona comes with primitive tools for dealing with coordinate reference systems, with GDAL library in the background

In [22]:
natural = fiona.open('../../data/protected_areas-etrs.shp', 'r')
from fiona.crs import to_string
print(to_string(natural.crs))

+init=epsg:4326


Simillary, new CRS definition can be created

In [23]:
from fiona.crs import from_epsg
from_epsg(3857)

{'init': 'epsg:3857', 'no_defs': True}

## Walking through features
we can either iterate through available features

In [25]:
for feature in chko[0:10]:
    print(feature['geometry']['type'])

Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon


or particular feature can be choosed

In [28]:
print(chko[54]['properties']['NAZEV'])

OrderedDict([('osm_id', '4704319'), ('name', 'Máchovo jezero'), ('type', 'water')])


## Feature geometry and shapely library
[Shapely](http://toblerity.org/shapely) converts feature geometry into GeoJSON structure. It also contains tools for geometry manipulations

In [29]:
from shapely.geometry import shape
cr = chko[54]
poly = shape(cr['geometry'])
poly.bounds

ImportError: No module named 'shapely'

Now we can either generalize given geometry or create buffer

In [ ]:
simple = poly.simplify(10)
simple.intersects(poly)
buff = poly.buffer(10)
buff.contains(poly)

some feature attributes can be fixed as well, at the end, we write new created feature down

In [ ]:
from shapely.geometry import mapping
feature = cr.copy()
feature.update(id=-1, geometry=mapping(buff))
feature["properties"].update(NAZEV="Mordor")
print(feature["properties"].keys())
print(chko.schema["properties"].keys())
chko = fiona.open('../../data/protected_areas-etrs.shp', 'a')
print("Features before: %d " % len(chko))
chko.write(feature)
print("Features after: %d " % len(chko))
chko.close()

---
[<- Introduction to vectors](00_introduction.ipynb) | [GDAL ->](02_ogr.ipynb)